In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("/workspaces/ask-cv/H. Versteeg, W. Malalasekra - An Introduction to Computational Fluid Dynamics_ The Finite Volume Method (2nd Edition)  -Prentice Hall (2007).pdf")
pages = loader.load_and_split()

In [4]:
pages[0]

Document(page_content='An Introduction\nto ComputationalFluid Dynamics\nTHE FINITE VOLUME METHOD\nSecond Edition\nH K Versteeg and W MalalasekeraANIN_A01.qxd  29/12/2006  09:53 AM  Page iii', metadata={'source': '/workspaces/ask-cv/H. Versteeg, W. Malalasekra - An Introduction to Computational Fluid Dynamics_ The Finite Volume Method (2nd Edition)  -Prentice Hall (2007).pdf', 'page': 0})

In [6]:
data = loader.load()

# Split the PDF into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(data)


In [9]:
from langchain_community.vectorstores import FAISS

In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/codespace/.python/current/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
db = FAISS.from_documents(texts, embeddings)

In [10]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma                     requires sqlite3 >= 3.35.0.[0m
[94mPlease visit                     https://docs.trychroma.com/troubleshooting#sqlite to learn how                     to upgrade.[0m

In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(texts, embedding_function)

# query it
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

In [7]:
embeddings = VertexAIEmbeddings()

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `VertexAIEmbeddings` was deprecated in LangChain 0.0.12 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAIEmbeddings`.
  warn_deprecated(


ImportError: Please, install or upgrade the google-cloud-aiplatform library: pip install google-cloud-aiplatform>=1.38.0

In [5]:



# Create a Chroma vector store
vectorstore = Chroma.from_documents(texts, embeddings)

# Save the vector store to disk
vectorstore.persist_to_disk("./chroma_db")

ValueError: File path /workspaces/ask-cv/H. Versteeg, W. Malalasekra - An Introduction to Computational Fluid Dynamics\_ The Finite Volume Method (2nd Edition) -Prentice Hall (2007).pdf is not a valid file or url